### Importando as bibliotecas

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Importando o** **DataSet**

In [4]:
# Carregue o arquivo .csv
base = pd.read_csv("/content/drive/MyDrive/Andre/ADA/train.csv", sep = ",")
base.head()

,REF_DATE,T;ARGET,VAR2,IDADE,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,...,VAR141,VAR142,VAR143,VAR144,VAR145,VAR146,VAR147,VAR148,VAR149,ID;;;;;
0,01/06/2017;00:00:00+00:00,0,M,34.137,NaN,RO,-8.808779,-63.878470,D,E,...,2680.289259,D,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,181755;;;;;
1,18/08/2017;00:00:00+00:00,0,M,40.447,NaN,PB,-7.146537,-34.926080,E,E,...,1777.7254690000002,E,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,287633;;;;;
2,30/06/2017;00:00:00+00:00,0,F,33.515,NaN,RS,-27.900178,-53.314035,NaN,E,...,1695.4949789999996,E,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6;.1,88015;;;
3,05/08/2017;00:00:00+00:00,1,F,25.797,NaN,BA,-12.948874,-38.451863,E,E,...,1399.037809,E,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,122576;;;;;
4,29/07/2017;00:00:00+00:00,0,F,54.074,NaN,RS,-30.051810,-51.213277,B,E,...,7868.793295999999,C,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE,2.6.1,1272;;;;;


# **Selecionando as colunas** **importantes**

In [5]:
base = base.rename(columns={'T;ARGET': 'TARGET'})
colunas_selecionadas = ['REF_DATE', 'TARGET', 'VAR2', 'IDADE', 'VAR4', 'VAR5', 'VAR8']
base = base[colunas_selecionadas]
#base.head()
#base.info()
#base.describe()

In [6]:
# Tratando os valores da variável VAR4(Flag de óbito):
# Substituir os valores 'S' por 1 e os valores NaN por 0
base['VAR4'] = base['VAR4'].replace('S', 1)  # Substituir 'S' por 1
base['VAR4'] = base['VAR4'].fillna(0)
base['VAR4'] = base['VAR4'].astype(int)


<ipython-input-6-3b41a592952a>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  base['VAR4'] = base['VAR4'].replace('S', 1)  # Substituir 'S' por 1


In [7]:
# Tratando os valores da variável VAR8(Classe social):
# Substituir os valores 'A' por 1, 'B' por 2, 'C' por 3, 'D' por 4 e 'E' por 5.
base['VAR8'] = base['VAR8'].replace('A', 1)  # Substituir 'A' por 1
base['VAR8'] = base['VAR8'].replace('B', 2)  # Substituir 'B' por 2
base['VAR8'] = base['VAR8'].replace('C', 3)  # Substituir 'C' por 3
base['VAR8'] = base['VAR8'].replace('D', 4)  # Substituir 'D' por 4
base['VAR8'] = base['VAR8'].replace('E', 5)  # Substituir 'E' por 5
base['VAR8'] = base['VAR8'].fillna(0)
base['VAR8'] = base['VAR8'].astype(int)
#base['VAR8']



<ipython-input-7-6d3f70e23904>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  base['VAR8'] = base['VAR8'].replace('E', 5)  # Substituir 'E' por 5


In [8]:
base['VAR8'] = base['VAR8'].replace(0, np.nan)
base['VAR4'] = base['VAR4'].replace(0, np.nan)

# **Tratando valores ausentes**

In [9]:
# Uso de Gaussiana para a Coluna IDADE. As colunas "REF_DATE" e "TARGET" não possuem valores ausentes.
# As colunas "Sexo" "VAR5: Unidade Federativa (UF)" apesar de possuirem valores ausentes, possuem dados categóricos, logo, a Gaussiana n é aplicável.
# As colunas "VAR4(Flag de óbito)" e "VAR8 (Classe social)" possuem uma alta proporção de valores ausentes. Gaussiana não é aplicável.
# Sugere-se tratar como uma categoria separada (ex.: "Desconhecido").

# Imputação da coluna IDADE com distribuição gaussiana
mean = base['IDADE'].mean()
std = base['IDADE'].std()
missing_count = base['IDADE'].isna().sum()
imputed_values = np.random.normal(loc=mean, scale=std, size=missing_count)
base.loc[base['IDADE'].isna(), 'IDADE'] = imputed_values

# Imputação para colunas categóricas com moda
categorical_columns_mode = ['VAR2', 'VAR5']
imputer_mode = SimpleImputer(strategy='most_frequent')
base[categorical_columns_mode] = imputer_mode.fit_transform(base[categorical_columns_mode])

In [10]:
# Função para imputação com regressão
def impute_with_regression(target_col, predictor_cols, dataset):
    # Separar dados completos e incompletos
    complete_data = base[base[target_col].notna()]
    incomplete_data = base[base[target_col].isna()]

    # Garantir que há dados suficientes para regressão
    if complete_data.shape[0] < 10 or incomplete_data.shape[0] == 0:
        return base

    # Definir modelo de regressão
    model = LinearRegression()

    # Treinar o modelo
    X_train = complete_data[predictor_cols]
    y_train = complete_data[target_col]
    model.fit(X_train, y_train)

    # Prever valores ausentes
    X_missing = incomplete_data[predictor_cols]
    predictions = model.predict(X_missing)

    # Imputar os valores ausentes no dataset
    base.loc[base[target_col].isna(), target_col] = predictions

    return base

In [12]:
# Imputação para VAR4 (Flag de óbito) usando regressão
predictor_columns_var4 = ['IDADE', 'TARGET']  # Ajuste preditores conforme necessário
base = impute_with_regression('VAR4', predictor_columns_var4, base)

# Imputação para VAR8 (Classe social) usando regressão
predictor_columns_var8 = ['IDADE', 'TARGET']  # Ajuste preditores conforme necessário
base = impute_with_regression('VAR8', predictor_columns_var8, base)

# Verificar valores ausentes após a imputação
missing_after_imputation = base.isna().sum()
print("Valores ausentes após a imputação:")
print(missing_after_imputation)

Valores ausentes após a imputação:
REF_DATE    0
TARGET      0
VAR2        0
IDADE       0
VAR4        0
VAR5        0
VAR8        0
dtype: int64


In [15]:
# Salvar o dataset processado
base.to_csv('/content/drive/MyDrive/Andre/ADA/dataset_imputado.csv', index=False)
print("Dataset salvo como 'dataset_imputado.csv'")

Dataset salvo como 'dataset_imputado.csv'
